## Data Processing Notebook
### Evolution of obesity rates in United States, over the last decade

This is a project work to demonstrate data analysis using Python. 

Obesity is a common, serious, and costly disease. Obesity-related conditions include heart disease, stroke, type 2 diabetes and certain types of cancer. These are among the leading causes of preventable, premature death. The estimated annual medical cost of obesity in the United States was nearly 173 billion in 2019 dollars. Medical costs for adults who had obesity were $1,861 higher than medical costs for people with healthy weight.

In this project we will explore Nutrition, Physical Activity, and Obesity - Behavioral Risk Factor Surveillance System data. We will also explore whether there is a strong correlation between obesity and age, gender, race, income and how these trends vary across United States.

**About Data**:
This dataset includes data on adult's diet, physical activity, and weight status from Behavioral Risk Factor Surveillance System. This data is used for DNPAO's Data, Trends, and Maps database, which provides national and state specific data on obesity, nutrition, physical activity, and breastfeeding.
https://chronicdata.cdc.gov/Nutrition-Physical-Activity-and-Obesity/Nutrition-Physical-Activity-and-Obesity-Behavioral/hn4x-zwk7

**Updated**: December 7, 2021

**Data Provided by**: Centers for Disease Control and Prevention (CDC), National Center for Chronic Disease Prevention and Health Promotion, Division of Nutrition, Physical Activity, and Obesity

Please see Project-Obesity-Analysis notebook for the conclusions. 

### Research Questions
1. Which state has highest precentage of obese adult population?
2. Are there any state and national trends related to obesity in adults? 
3. Are there differences in obesity rates across Gender, Age, Race, Income and Education?
4. Does low levels of exercise and poor nutrition correlate with higher levels of obesity?

**GitHub:** https://github.com/nsharma73/python_data_analysis


In [1]:
# Import key libraries for analysis
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px
from IPython.display import display

Read data from csv. This is data can be downloaded from CDC website as discussed in the intro.

In [2]:
df = pd.read_csv('Nutrition_Physical_Activity_and_Obesity.csv')

Ensure data is loaded correctly and total row count is as expected.

In [3]:
original_row_count = len(df.index)
df.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,Datasource,Class,Topic,Question,Data_Value_Unit,Data_Value_Type,...,GeoLocation,ClassID,TopicID,QuestionID,DataValueTypeID,LocationID,StratificationCategory1,Stratification1,StratificationCategoryId1,StratificationID1
0,2014,2014,GU,Guam,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(13.444304, 144.793731)",OWS,OWS1,Q036,VALUE,66,Education,High school graduate,EDU,EDUHSGRAD
1,2013,2013,US,National,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,NaN,OWS,OWS1,Q036,VALUE,59,Income,"$50,000 - $74,999",INC,INC5075
2,2013,2013,US,National,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,NaN,OWS,OWS1,Q037,VALUE,59,Income,Data not reported,INC,INCNR
3,2015,2015,US,National,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who achieve at least 300 min...,NaN,Value,...,NaN,PA,PA1,Q045,VALUE,59,Income,"Less than $15,000",INC,INCLESS15
4,2015,2015,GU,Guam,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who achieve at least 150 min...,NaN,Value,...,"(13.444304, 144.793731)",PA,PA1,Q044,VALUE,66,Race/Ethnicity,Hispanic,RACE,RACEHIS


In [4]:
df.describe(include='all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
YearStart,80929.0,NaN,NaN,NaN,2015.536717,2.841623,2011.0,2013.0,2016.0,2018.0,2020.0
YearEnd,80929.0,NaN,NaN,NaN,2015.536717,2.841623,2011.0,2013.0,2016.0,2018.0,2020.0
LocationAbbr,80929,55,CO,1512,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LocationDesc,80929,55,Colorado,1512,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Datasource,80929,1,Behavioral Risk Factor Surveillance System,80929,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Class,80929,3,Physical Activity,44805,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Topic,80929,3,Physical Activity - Behavior,44805,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Question,80929,9,Percent of adults aged 18 years and older who ...,15037,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Data_Value_Unit,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Data_Value_Type,80929,1,Value,80929,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Exploration 
This is a survey data so we will start by looking at questions

In [5]:
qs = set(df['Question'])

In [6]:
qs

{'Percent of adults aged 18 years and older who have an overweight classification',
 'Percent of adults aged 18 years and older who have obesity',
 'Percent of adults who achieve at least 150 minutes a week of moderate-intensity aerobic physical activity or 75 minutes a week of vigorous-intensity aerobic activity (or an equivalent combination)',
 'Percent of adults who achieve at least 150 minutes a week of moderate-intensity aerobic physical activity or 75 minutes a week of vigorous-intensity aerobic physical activity and engage in muscle-strengthening activities on 2 or more days a week',
 'Percent of adults who achieve at least 300 minutes a week of moderate-intensity aerobic physical activity or 150 minutes a week of vigorous-intensity aerobic activity (or an equivalent combination)',
 'Percent of adults who engage in muscle-strengthening activities on 2 or more days a week',
 'Percent of adults who engage in no leisure-time physical activity',
 'Percent of adults who report consum

#### Explore non-numeric data to better understand the categorical variables
We will create a list of columns and then pass this list to a function, 
which will create a set of unique values for each attribute

In [7]:
col_dtype_n = pd.DataFrame(df.describe().transpose()).index
col_names = pd.DataFrame(df.columns)
print(col_dtype_n)
print(col_names)

Index(['YearStart', 'YearEnd', 'Data_Value_Unit', 'Data_Value',
       'Data_Value_Alt', 'Low_Confidence_Limit', 'High_Confidence_Limit ',
       'LocationID'],
      dtype='object')
                             0
0                    YearStart
1                      YearEnd
2                 LocationAbbr
3                 LocationDesc
4                   Datasource
5                        Class
6                        Topic
7                     Question
8              Data_Value_Unit
9              Data_Value_Type
10                  Data_Value
11              Data_Value_Alt
12  Data_Value_Footnote_Symbol
13         Data_Value_Footnote
14        Low_Confidence_Limit
15      High_Confidence_Limit 
16                 Sample_Size
17                       Total
18                  Age(years)
19                   Education
20                      Gender
21                      Income
22              Race/Ethnicity
23                 GeoLocation
24                     ClassID
25         

In [8]:
str_col = col_names[pd.isna(col_names[col_names.isin(col_dtype_n)])].dropna(axis=0)

In [9]:
i_cols_array = np.array(str_col)
# Let's test whether the data are appropriately organized in the set with unique values 
# we are testing the code with questions set
set(np.array(df[['Question']])[:,0])

{'Percent of adults aged 18 years and older who have an overweight classification',
 'Percent of adults aged 18 years and older who have obesity',
 'Percent of adults who achieve at least 150 minutes a week of moderate-intensity aerobic physical activity or 75 minutes a week of vigorous-intensity aerobic activity (or an equivalent combination)',
 'Percent of adults who achieve at least 150 minutes a week of moderate-intensity aerobic physical activity or 75 minutes a week of vigorous-intensity aerobic physical activity and engage in muscle-strengthening activities on 2 or more days a week',
 'Percent of adults who achieve at least 300 minutes a week of moderate-intensity aerobic physical activity or 150 minutes a week of vigorous-intensity aerobic activity (or an equivalent combination)',
 'Percent of adults who engage in muscle-strengthening activities on 2 or more days a week',
 'Percent of adults who engage in no leisure-time physical activity',
 'Percent of adults who report consum

In [10]:
for i in i_cols_array:
    print(i)
    print(set(np.array(df[i])[:,0]))

['LocationAbbr']
{'NH', 'DE', 'OH', 'OK', 'NE', 'IA', 'NM', 'PA', 'ND', 'AK', 'CA', 'ID', 'ME', 'MD', 'MI', 'SD', 'CO', 'OR', 'MN', 'NY', 'KS', 'IN', 'NV', 'UT', 'TN', 'GA', 'MA', 'VA', 'WY', 'PR', 'MS', 'NC', 'RI', 'GU', 'VI', 'IL', 'WA', 'MT', 'WI', 'SC', 'NJ', 'VT', 'KY', 'AZ', 'TX', 'DC', 'LA', 'CT', 'HI', 'AL', 'AR', 'FL', 'MO', 'WV', 'US'}
['LocationDesc']
{'Wisconsin', 'Delaware', 'Montana', 'Oklahoma', 'Virgin Islands', 'New Hampshire', 'Maryland', 'Kansas', 'District of Columbia', 'Utah', 'Massachusetts', 'South Carolina', 'Indiana', 'Connecticut', 'New York', 'Arkansas', 'Oregon', 'Alaska', 'North Dakota', 'Nevada', 'Ohio', 'Guam', 'Louisiana', 'Puerto Rico', 'South Dakota', 'Pennsylvania', 'New Jersey', 'Virginia', 'North Carolina', 'Alabama', 'Iowa', 'Idaho', 'Colorado', 'New Mexico', 'Texas', 'West Virginia', 'Michigan', 'Rhode Island', 'Kentucky', 'Georgia', 'Vermont', 'Nebraska', 'Mississippi', 'Maine', 'California', 'Arizona', 'Missouri', 'Florida', 'Illinois', 'Wyoming

In [11]:
df[['LocationID','LocationDesc']].head(15)

,LocationID,LocationDesc
0,66,Guam
1,59,National
2,59,National
3,59,National
4,66,Guam
5,56,Wyoming
6,11,District of Columbia
7,72,Puerto Rico
8,1,Alabama
9,66,Guam


### Key Takeaways
1. LocationAbbr and LocationDesc are state codes and state names, we can drop LocationID
2. Datasource has only one value "Behavioral Risk Factor Surveillance System" and can be deleted
3. Class and Topic work together: both are important and provide data about the question category
    'Fruits and Vegetables', 'Obesity / Weight Status', 'Physical Activity' for class, and
    'Physical Activity - Behavior', 'Fruits and Vegetables - Behavior', 'Obesity / Weight Status' for topic
4. ClassID and TopicID may not be needed
5. There are 9 questions and these are key to this analysis and we may not need QuestionID
6. DataValueType and DataValueType are not needed
7. Data_Value_Footnote_Symbol is not needed
8. Data_Value_Footnote should be used to delete rows where sample size is not sufficient 
9. Sample_Size should be converted to numeric data type
10. Total column can be used to get totals without any stratification, we will use StratificationCategory1 instead
11. Age, Eduction, Gender, Income, Race/Ethnicity are key dimensions for this analysis
12. We will not need GeoLocation, and we will work with State data
13. Stratification1 is more granular than StratificationCategory1; Hierarchical Data
14. StratificationCategoryId1 and StratificationID1 are not needed


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80929 entries, 0 to 80928
Data columns (total 33 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   YearStart                   80929 non-null  int64  
 1   YearEnd                     80929 non-null  int64  
 2   LocationAbbr                80929 non-null  object 
 3   LocationDesc                80929 non-null  object 
 4   Datasource                  80929 non-null  object 
 5   Class                       80929 non-null  object 
 6   Topic                       80929 non-null  object 
 7   Question                    80929 non-null  object 
 8   Data_Value_Unit             0 non-null      float64
 9   Data_Value_Type             80929 non-null  object 
 10  Data_Value                  72965 non-null  float64
 11  Data_Value_Alt              72965 non-null  float64
 12  Data_Value_Footnote_Symbol  7964 non-null   object 
 13  Data_Value_Footnote         796

### Data Cleaning
The strategy to clean data includes following tasks:
- Understand the data quality by exploring unique and null values
- Remove columns that are redundant or not needed
- Assess missing data rationale and handle missing data
- Certain columns appear to have only one value
- Rename columns to be more intuitive 

In [13]:
# The data value and alternative data values appear to be the same
sum(df.Data_Value_Alt != df.Data_Value)

7964

In [14]:
# Double check the data values before dropping the columns 
print(df.Data_Value_Unit.unique())
print(df.Data_Value_Type.unique())
print(df.Data_Value_Footnote_Symbol.unique())
print(df.Total.unique())
print(df.DataValueTypeID.unique())
print(df.Datasource.unique())
print(df.Data_Value_Footnote.unique())

[nan]
['Value']
[nan '~']
[nan 'Total']
['VALUE']
['Behavioral Risk Factor Surveillance System']
[nan 'Data not available because sample size is insufficient.']


In [15]:
df.drop(['Data_Value_Unit', 'Data_Value_Type', 'LocationID', 'GeoLocation',
         'Data_Value_Alt', 'Data_Value_Footnote_Symbol','Total',
         'DataValueTypeID','Datasource'], axis=1, inplace = True)

In [16]:
len(df.index)

80929

#### Delete records where 'Data not available because sample size is insufficient.'

In [17]:
df.drop(df[df['Data_Value_Footnote'] == 
           'Data not available because sample size is insufficient.'].index , inplace = True)

In [18]:
print(f'{round((original_row_count - len(df.index))/original_row_count*100,2)}\
 percent was deleted due to insifficient sample size')

9.84 percent was deleted due to insifficient sample size


In [19]:
len(df.index)

72965

#### The Data_Value_Footnote can be deleted now and we can simplify Year dimension

In [20]:
print(df.Data_Value_Footnote.unique())
# Data_Value_Footnote column is no longer needed, we used this data to delete missing values
df.drop(['Data_Value_Footnote'], axis=1, inplace = True)

[nan]


In [21]:
# The year start and end have same values, this is likely due to data collection period was within the year.
print(sum(df.YearStart != df.YearEnd))
# We do not need both columns, so we will keep Year End
df.drop(['YearStart'], axis=1, inplace = True)

0


#### The sample size will be used for weighted average, so we convert it to number

In [22]:
df['Sample_Size'] = df['Sample_Size'].str.replace(',', '').astype(float)

In [23]:
df.describe(include='all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
YearEnd,72965.0,NaN,NaN,NaN,2015.522415,2.849482,2011.0,2013.0,2016.0,2018.0,2020.0
LocationAbbr,72965,55,US,1512,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LocationDesc,72965,55,National,1512,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Class,72965,3,Physical Activity,40396,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Topic,72965,3,Physical Activity - Behavior,40396,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Question,72965,9,Percent of adults who engage in no leisure-tim...,13596,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Data_Value,72965.0,NaN,NaN,NaN,31.238672,10.156464,0.9,24.4,31.1,36.9,77.6
Low_Confidence_Limit,72965.0,NaN,NaN,NaN,26.933101,9.960837,0.3,20.1,26.7,32.8,70.2
High_Confidence_Limit,72965.0,NaN,NaN,NaN,36.109866,11.084842,3.0,28.6,35.8,42.1,87.7
Sample_Size,72965.0,NaN,NaN,NaN,3680.188995,18802.289294,50.0,527.0,1127.0,2418.0,476876.0


#### Questions are important data, these questions provide context to data value. There are 9 unique questions
The questions appear to quantify data value in terms of:
- What percentage of adults have obesity? or
- What percentage of adults exercise?

In [24]:
print(len(df.Question.unique()))
df.Question.unique()

9


array(['Percent of adults aged 18 years and older who have obesity',
       'Percent of adults aged 18 years and older who have an overweight classification',
       'Percent of adults who achieve at least 300 minutes a week of moderate-intensity aerobic physical activity or 150 minutes a week of vigorous-intensity aerobic activity (or an equivalent combination)',
       'Percent of adults who achieve at least 150 minutes a week of moderate-intensity aerobic physical activity or 75 minutes a week of vigorous-intensity aerobic physical activity and engage in muscle-strengthening activities on 2 or more days a week',
       'Percent of adults who engage in no leisure-time physical activity',
       'Percent of adults who engage in muscle-strengthening activities on 2 or more days a week',
       'Percent of adults who report consuming fruit less than one time daily',
       'Percent of adults who achieve at least 150 minutes a week of moderate-intensity aerobic physical activity or 75 mi

### We use a function to create a "status" column that represents questions
This step is data transformation, which may help us with readability and plotting the data

In [25]:
def qf_update(row):
    if row['Question'] == 'Percent of adults aged 18 years and older who have obesity':
        val = 'Obese'
    elif row['Question'] == 'Percent of adults aged 18 years and older who have an overweight classification':
        val = 'Overweight'
    elif row['Question'] == 'Percent of adults who achieve at least 300 minutes a week of moderate-intensity aerobic physical activity or 150 minutes a week of vigorous-intensity aerobic activity (or an equivalent combination)':
        val = 'Very Active'
    elif row['Question'] == 'Percent of adults who achieve at least 150 minutes a week of moderate-intensity aerobic physical activity or 75 minutes a week of vigorous-intensity aerobic physical activity and engage in muscle-strengthening activities on 2 or more days a week':
        val = 'Active and Physical Training' 
    elif row['Question'] == 'Percent of adults who achieve at least 150 minutes a week of moderate-intensity aerobic physical activity or 75 minutes a week of vigorous-intensity aerobic activity (or an equivalent combination)':
        val = 'Active'     
    elif row['Question'] == 'Percent of adults who engage in no leisure-time physical activity':
        val = 'Inactive'  
    elif row['Question'] == 'Percent of adults who engage in muscle-strengthening activities on 2 or more days a week':
        val = 'Physical Training'
    elif row['Question'] == 'Percent of adults who report consuming fruit less than one time daily':
        val = 'Fruits Deficient'
    elif row['Question'] == 'Percent of adults who report consuming vegetables less than one time daily':
        val = 'Veggies Deficient'
    else:
        val = 'no data available'
    return val

In [26]:
# Run the function, which will create a new column called "status"
df['status'] = df.apply(qf_update, axis=1)

### The data may be imbalanced due to the independence of questions and timing of surveys

In [27]:
df.status.value_counts()

Inactive                        13596
Obese                           13579
Overweight                      13579
Physical Training                6712
Active                           6700
Very Active                      6696
Active and Physical Training     6692
Fruits Deficient                 2708
Veggies Deficient                2703
Name: status, dtype: int64

In [28]:
# The following columns are not needed for analysis 
df.drop(['Class', 'Topic', 'Question','ClassID','TopicID','QuestionID',], axis=1, inplace=True)

In [29]:
# Renaming columns for ease of use during the code
df.rename(columns={'Data_Value': 'Percent_Adults', 
                   'Low_Confidence_Limit': 'LowCI', 
                   'High_Confidence_Limit ': 'HighCI' }, inplace=True)


### The categories and subcategories are key dimensions that are required to answer research questions
The category and subcategory provide us further context of data, for example age is the category and various ranges of ages such as 18 to 24 years is a subcategory of age. We will use these dimensions to compare obesity rates within each category such as age groups or male vs female obesity rates.

In [30]:
df.groupby(['StratificationCategory1','Stratification1'])['status'].count()

StratificationCategory1  Stratification1                 
Age (years)              18 - 24                             2878
                         25 - 34                             2878
                         35 - 44                             2878
                         45 - 54                             2878
                         55 - 64                             2878
                         65 or older                         2878
Education                College graduate                    2878
                         High school graduate                2878
                         Less than high school               2878
                         Some college or technical school    2878
Gender                   Female                              2878
                         Male                                2878
Income                   $15,000 - $24,999                   2878
                         $25,000 - $34,999                   2878
                  

In [31]:
print(df.StratificationCategory1.value_counts())
print(df.StratificationCategoryId1.value_counts())
print(df.Stratification1.value_counts())
print(df.StratificationID1.value_counts())

Income            20146
Age (years)       17268
Race/Ethnicity    15405
Education         11512
Gender             5756
Total              2878
Name: StratificationCategory1, dtype: int64
INC      20146
AGEYR    17268
RACE     15405
EDU      11512
GEN       5756
OVR       2878
Name: StratificationCategoryId1, dtype: int64
High school graduate                2878
Female                              2878
65 or older                         2878
$75,000 or greater                  2878
$35,000 - $49,999                   2878
College graduate                    2878
Male                                2878
35 - 44                             2878
Total                               2878
$15,000 - $24,999                   2878
$50,000 - $74,999                   2878
18 - 24                             2878
45 - 54                             2878
Some college or technical school    2878
55 - 64                             2878
25 - 34                             2878
$25,000 - $34,999   

In [32]:
# The ID fields do not provide additional information, so we will work with category and subcategories 
df.drop(['StratificationCategoryId1', 'StratificationID1'], axis=1, inplace=True)

In [33]:
# renaming the columns for ease of use
df.rename(columns={'StratificationCategory1': 'Category', 
                   'Stratification1': 'Sub_Category',
                   'YearEnd':'Year',
                   'LocationAbbr':'State',
                   'LocationDesc':'State_Name',
                   'Age(years)':'Age',
                   'Race/Ethnicity':'Race'
                  }, inplace=True)

### The columns Age, Edcation, Gender, Income, Race/Ethnicity have null values
These columns are not fully populated because the data in these columns is only populated when relevant, for example Gender column is not relevant when the row is populated for a particular age group or race.

In [34]:
df.isnull().sum(axis = 0)

Year                  0
State                 0
State_Name            0
Percent_Adults        0
LowCI                 0
HighCI                0
Sample_Size           0
Age               55697
Education         61453
Gender            67209
Income            52819
Race              57560
Category              0
Sub_Category          0
status                0
dtype: int64

### Let's explore location data

In [35]:
# the data here is represented at state and national level
df.State.unique()

array(['GU', 'US', 'WY', 'DC', 'PR', 'AL', 'RI', 'NJ', 'WA', 'MI', 'VA',
       'CA', 'UT', 'NY', 'MA', 'DE', 'AR', 'IL', 'NH', 'NM', 'MD', 'HI',
       'LA', 'TX', 'SD', 'CO', 'OK', 'MS', 'OR', 'WV', 'WI', 'KS', 'FL',
       'ID', 'AZ', 'VI', 'MT', 'MN', 'GA', 'NC', 'PA', 'KY', 'ND', 'SC',
       'NE', 'MO', 'NV', 'IA', 'IN', 'OH', 'VT', 'TN', 'CT', 'AK', 'ME'],
      dtype=object)

In [36]:
st_lookup = pd.read_csv('states.csv')

In [37]:
st_lookup_dict = dict(zip(st_lookup.State, st_lookup.Region))

In [47]:
st_lookup_dict

{'AK': 'Pacific',
 'AL': 'East South Central',
 'AR': 'West South Central',
 'AZ': 'Mountain',
 'CA': 'Pacific',
 'CO': 'Mountain',
 'CT': 'New England',
 'DC': 'South Atlantic',
 'DE': 'South Atlantic',
 'FL': 'South Atlantic',
 'GA': 'South Atlantic',
 'HI': 'Pacific',
 'IA': 'West North Central',
 'ID': 'Mountain',
 'IL': 'East North Central',
 'IN': 'East North Central',
 'KS': 'West North Central',
 'KY': 'East South Central',
 'LA': 'West South Central',
 'MA': 'New England',
 'MD': 'South Atlantic',
 'ME': 'New England',
 'MI': 'East North Central',
 'MN': 'West North Central',
 'MO': 'West North Central',
 'MS': 'East South Central',
 'MT': 'Mountain',
 'NC': 'South Atlantic',
 'ND': 'West North Central',
 'NE': 'West North Central',
 'NH': 'New England',
 'NJ': 'Middle Atlantic',
 'NM': 'Mountain',
 'NV': 'Mountain',
 'NY': 'Middle Atlantic',
 'OH': 'East North Central',
 'OK': 'West South Central',
 'OR': 'Pacific',
 'PA': 'Middle Atlantic',
 'RI': 'New England',
 'SC': 'Sout

In [38]:
len(st_lookup_dict)

51

In [39]:
st_lookup_dict.update(GU ='Other') 
st_lookup_dict.update(PR ='Other') 
st_lookup_dict.update(US ='Other') 
st_lookup_dict.update(VI ='Other') 

In [40]:
df['Region'] = df['State'].apply(lambda x : st_lookup_dict[x])

In [41]:
us_df = df.loc[df['State_Name'] == 'National']
st_df = df.loc[df['State_Name'] != 'National']

In [42]:
print('National data: ',len(us_df))
print('State data: ',len(st_df))
print('Total data: ',len(us_df)+len(st_df))

National data:  1512
State data:  71453
Total data:  72965


In [43]:
df.describe(include='all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Year,72965.0,NaN,NaN,NaN,2015.522415,2.849482,2011.0,2013.0,2016.0,2018.0,2020.0
State,72965,55,US,1512,NaN,NaN,NaN,NaN,NaN,NaN,NaN
State_Name,72965,55,National,1512,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Percent_Adults,72965.0,NaN,NaN,NaN,31.238672,10.156464,0.9,24.4,31.1,36.9,77.6
LowCI,72965.0,NaN,NaN,NaN,26.933101,9.960837,0.3,20.1,26.7,32.8,70.2
HighCI,72965.0,NaN,NaN,NaN,36.109866,11.084842,3.0,28.6,35.8,42.1,87.7
Sample_Size,72965.0,NaN,NaN,NaN,3680.188995,18802.289294,50.0,527.0,1127.0,2418.0,476876.0
Age,17268,6,25 - 34,2878,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Education,11512,4,High school graduate,2878,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gender,5756,2,Female,2878,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Let's explore how data is structed by year

In [44]:
print(df.Year.value_counts())
print(len(us_df))
print(len(st_df))

2017    12329
2019    12051
2015     9497
2011     9307
2013     9257
2016     4177
2020     4118
2018     4115
2014     4114
2012     4000
Name: Year, dtype: int64
1512
71453


#### The data is ready for analysis
Clean data will be split by national (USA) level and state level into two csv files.

In [45]:
us_df.to_csv('us_df.csv', sep=',', index=False, encoding='utf-8')
st_df.to_csv('st_df.csv', sep=',', index=False, encoding='utf-8')

In [46]:
print('End of Data Clean Up')

End of Data Clean Up
